# Динамическое программирование

*Динамическое программирование* — это метод, при котором мы формулируем задачу через набор подзадач, выясняем зависимости между ними и последовательно заполняем таблицу результатов, используя уже найденные значения для вычисления следующих.

## Лесенка

### Базовая задача

Внизу лестницы стоит кролик, который умеет прыгать на следующую ступеньку, через ступеньку и через 2 ступеньки. Наверху лестинцы кролика ждет удав. Требуется найти количество путей, которыми кролик сможет добраться до удава

<img src='img/dp1.jpg'>

- Очевидно, что на первую ступеньку можно придти лишь одним способом
- На вторую ступеньку можно придти 2 способами: либо мы снчала прыгаем на первую ступеньку, а затем на вторую, либо сразу прыгаем через первую ступеньку
- На третью ступеньку можно придти так:
    - либо мы снчала прыгаем на первую ступеньку, затем на вторую, потом на третью
    - либо прыгаем через первую ступеньку на вторую, а потом прыгаем на третью
    - либо последовательно прыгаем на первую, вторую, третью ступеньки
    - либо сразу прыгаем на третью ступеньку с первой

<img src='img/dp2.jpg'>

Сколькими способами можно придти на 4-ю ступеньку?
- Во-первых, мы знаем, что на третью ступеньку мы могли придти 4 способами, а значит можно "продолжить" каждый из этих способов, просто добавив 1 прыжок
- Во-вторых, мы можем прыгнуть со второй ступеньки. На вторую ступеньку ведет 2 пути, значит с каждого из них можно сделать прыжок
- В-третьх, мы можем прыгнуть с первой ступеньки. На нее ведет лишь 1 путь, поэтому к общему количеству способов попасть на 4-ю ступеньку добавляем 1 

Итого: $4+2+1=7$ способов попасть на 4-ю ступень

Рассуждаем аналогично для 5-й ступеньки: в нее мы могли попасть только со 2, 3 и 4 ступеней. Значит, количество способов попасть на 5-ю ступень равно сумме способов попасть на 2, 3 и 4 ступеньку — $2+4+7=13$

Эту лесенку можно представить в виде массива `dp=[0...n-1]`. Пусть изначально он заполнен нулями.

В первую очередь нужно заполнить *базу динамики*, то есть записать в `dp` базовые случаи — количество способов попасть на первую, вторую и третью ступеньки.

Далее - как мы поняли, на каждую следующую ступеньку можно попасть суммируя количество способов попасть на предыдущие три

In [6]:
n = 7
dp = [0] * (n)
dp[0] = dp[1] = 1
dp[2] = 2

for i in range(3, n):
    dp[i] = dp[i-1] + dp[i-2] + dp[i-3]

print(max(dp))

[1, 1, 2, 4, 7, 13, 24]


### Платная лестница

Пусть теперь за каждую ступеньку взимается плата $x\in\mathbb{Z}$. Мы можем прыгать на 1 ступеньку или на 2. Наша задача достигнуть вершины лестицы заплатив как можно меньше (а возможно и оказаться в плюсе)

Зелёным отмечена цена каждой ступеньки
<img src='img/dp3.jpg'>

- Чтобы придти на первую и вторую ступеньку нужно заплатить 4 и 7 д.е. соответственно. Это база динамики
- На третью ступеньку можно попасть
    - с первой, тогда мы заплатим $4-5=-1$ (получается нам заплатят)
    - со второй, тогда мы заплатим $7-5=2$
- Мы, конечно, выбираем первый вариант, ведь мы получаем деньги
- На 4-ю ступень можно попасть
    - с третьей, тогда в сумме мы потратим $-1+1=0$
    - со второй, тогда заплатим $7+1=8$
- Выберем первый вариант
- Аналогично для оставшихся ступеней

Синим отметим наш оптимальный выбор на каждой ступени

<img src='img/dp4.jpg'>

In [10]:
n = 7
a = [0, 4, 7, -5, 1, 5, 15, 0]
dp = [0] * 101
dp[0] = 0
dp[1] = a[1]

for i in range(2, n + 1):
    dp[i] = a[i] + min(dp[i-1], dp[i-2])

print(dp[n])

4


## Два параметра

Допустим, у нас есть квадратное поле $n\times n$, состоящее из клеток. Какие-то клетки недостижимы и по ним ходить нельзя. 

Разрешённые ходы: вправо и вниз. Нужно найти, сколькими способами можно попасть из клетки $(0, 0)$ в клетку $(n−1, n−1)$

<img src='img/dp10.jpg'>

Для каждой клетки мы хотим знать - сколько путей ведёт к ней из старта, учитывая ограничения.

Пусть `dp[i][j]` - количество способов попасть в клетку $(i, j)$. Значит, если клетка достижима, то в нее можно придти либо сверху - из клетки $(i-1, j)$, либо слева - из $(i, j-1)$, то есть
$$dp[i][j]=dp[i-1][j]+dp[i][j-1]$$
Если клетка нелостижима, то ставим `dp[i][j]=0`

Особым случаем будут являться первая строка и первый столбец, так как в них в клетки можно придти только слева и справа соответственно

In [12]:
n = 5
grid = [
    '...#.',
    '.#.##',
    '....#',
    '#.#.#',
    '.....'
]

dp = [[0] * n for _ in range(n)]
dp[0][0] = 1

for j in range(1, n):
    if grid[0][j] == '.':
        dp[0][j] = dp[0][j - 1]

for i in range(1, n):
    if grid[i][0] == '.':
        dp[i][0] = dp[i - 1][0]

for i in range(1, n):
    for j in range(1, n):
        if grid[i][j] == '.':
            dp[i][j] = dp[i - 1][j] + dp[i][j - 1]

print(dp[n - 1][n - 1])


3
[[1, 1, 1, 0, 0], [1, 0, 1, 0, 0], [1, 1, 2, 2, 0], [0, 1, 0, 2, 0], [0, 1, 1, 3, 3]]


На рисунке отмечены все пути, по которым можно добраться до клетки $(n-1, n-1)$
<img src='img/dp11.jpg'>

## Наибольшая возрастающая подпоследовательность

Дана последовательность чисел `a = [5, 7, 5, 1, 3, 4, 2, 1, 8, 4, 6, 3, 6]`. Наша задача выбрать максимальное число элементов (слева направо), чтобы образованная ими последовательность возрастала

Создадим массив `dp` и заполним его единицами, то есть каждый элемент является возрастающей последовательностью длины 1. Тогда, мы будем идти по массиву и смотреть, к какой уже существующей последовательности можно дописать текущий элемент

<img src='img/dp5.jpg'>

- Начинаем с `arr[1]=7`: элемент можно дописать к `arr[0]=5`. Тогда получится возрастающая последовательность (далее - ВП) длины 2. Ставим `dp[1]=2`
- Смотрим на `arr[2]=5`: этот элемент нельзя дописать ни к `arr[0]`, ни к `arr[1]`, так чтобы получилась ВП
- Аналогично, нельзя дописать `arr[3]=1` - не получится дополнить ВП
- Рассмотрим `arr[4]=3`: это число можно дописать к `arr[3]=1` - получится ВП длины 2, значит ставим `dp[4]=2`
- Теперь `arr[5]=4`: этот элемент можно дописать к `arr[4]=3` - получится ВП длины 3, так как имеем ВП `1-3-4`. Значит, `dp[5]=3`
- Смотрим на элемент `arr[6]=2`: его можно дописать только к `arr[3]=1` - получится ВП длины 2, значит ставим `dp[6]=2`

Продолжая, получим, что массив `dp` будет выглядеть так

<img src='img/dp6.jpg'>

Ответом на вопрос задачи - "какова длина НВП" будет `max(dp)`. В данном случае это 4

Подходящие подпоследовательности, например
- $1\to3\to4\to8$
- $1\to2\to4\to6$
- $1\to3\to4\to6$

Но как нам определить не только длину НВП, но и саму подпоследовтальность? Для этого нужно создать массив предков `p`. В этом массиве мы будем запоминать из какого элемента мы пришли в текущий. Таким образом мы построим дерево, где корнем будет первый элемент массива, а дальше - те вершины в которых продолжается ВП

Обозначим корень как `p[0]=-1`. Далее
- если элементом нельзя продолжить ВП будем ставить `-1`
- если можно - ставим индекс вершины, из которой мы пришли к текущему элементу

Следуя этой логике для данного `arr` массив предков примет вид:

<img src='img/dp8.jpg'>

Теперь можем построить наш граф: черным отмечены индексы, синим - значения из `arr`

<img src='img/dp9.jpg'>

In [6]:
arr = [5, 7, 5, 1, 3, 4, 2, 1, 8, 4, 6, 3, 6]
dp = [1] * len(arr)
p = [-1] * len(arr)

for i in range(1, len(dp)):
    for j in range(0, i):
        if arr[i] > arr[j] and dp[i] < dp[j] + 1:
            dp[i] = dp[j] + 1
            p[i] = j

print(max(dp))

v = dp.index(max(dp))
while v != -1:
    print(arr[v], end = ' ')
    v = p[v]

4
8 4 3 1 

Такой алгоритм работает за $O(n^2)$ (тут рил очев почему)